# Setting up AB testing on multiple models with production variants

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r model_a_s3_path
%store -r model_b_s3_path

In [ ]:
%store -r s3_bucket
%store -r prefix

In [ ]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()

In [ ]:
from sagemaker.image_uris import retrieve

image_uri = retrieve(
    "xgboost", 
    region="us-east-1", 
    version="0.90-2"
)

image_uri

In [ ]:
image_uri_a = image_uri
image_uri_b = image_uri

In [ ]:
container1 = { 
    'Image': image_uri_a,
    'ContainerHostname': 'containerA',
    'ModelDataUrl': model_a_s3_path
}

container2 = { 
    'Image': image_uri_b,
    'ContainerHostname': 'containerB',
    'ModelDataUrl': model_b_s3_path
}

In [ ]:
model_name = "ab-testing"
endpoint_config_name = 'ab-testing-config'
endpoint_name = 'ab-testing-endpoint'

In [ ]:
import boto3
sm_client = boto3.Session().client('sagemaker')

In [ ]:
model_name_a = "ab-model-a"
model_name_b = "ab-model-b"
endpoint_config_name = 'ab-endpoint-config'
endpoint_name = 'ab-endpoint'

In [ ]:
try:
    sm_client.delete_model(ModelName=model_name_a)
    sm_client.delete_model(ModelName=model_name_b)
except:
    pass

In [ ]:
response = sm_client.create_model(
    ModelName        = model_name_a,
    ExecutionRoleArn = role,
    Containers       = [container1])
print(response)

response = sm_client.create_model(
    ModelName        = model_name_b,
    ExecutionRoleArn = role,
    Containers       = [container2])
print(response)

In [ ]:
from sagemaker.session import production_variant

variant1 = production_variant(
    model_name=model_name_a,
    instance_type="ml.t2.medium",
    initial_instance_count=1,
    variant_name='VariantA',
    initial_weight=0.5
)
                              
variant2 = production_variant(
    model_name=model_name_b,
    instance_type="ml.t2.medium",
    initial_instance_count=1,
    variant_name='VariantB',
    initial_weight=0.5
)

In [ ]:
session.endpoint_from_production_variants(
    name=endpoint_name,
    production_variants=[variant1, variant2]
)

In [ ]:
runtime_sm_client = boto3.client('sagemaker-runtime')

In [ ]:
body = "10,-5"

In [ ]:
from time import sleep

def test_ab_testing_setup():
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='text/csv',
        Body=body
    )
    
    variant = response['InvokedProductionVariant']
    b = response['Body'].read()
    prediction = b.decode("utf-8")

    print(variant + " - "+ prediction)

for _ in range(0,10):
    test_ab_testing_setup()
    sleep(1)

In [ ]:
def test_direct_call():
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='text/csv',
        TargetVariant='VariantB',
        Body=body
    )

    variant = response['InvokedProductionVariant']
    b = response['Body'].read()
    prediction = b.decode("utf-8")

    print(variant + " - "+ prediction)

for _ in range(0,10):
    test_direct_call()
    sleep(1)

In [ ]:
response = sm_client.delete_endpoint(
    EndpointName=endpoint_name
)

response